In [1]:
!pip install opacus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.4/254.4 kB 4.8 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
from opacus import PrivacyEngine
import time
import matplotlib.pyplot as plt

# Select Dataset
All dataset arguments default to MNIST. This goes for the `run_experiment` function later. It is important to run `select_dataset` at least once as this loads the data. If you opt to run all cells, it will load MNIST by default.

In [3]:
def select_dataset(name='MNIST'):
  if name == "MNIST":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),])

    full_train = datasets.MNIST(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 50000
    val_size = len(full_train) - train_size
    train_dataset, val_dataset = random_split(full_train, [train_size, val_size])

    test_dataset = datasets.MNIST(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )

  elif name == "CIFAR":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])


    full_train = datasets.CIFAR10(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 40000
    val_size = len(full_train) - train_size
    train_dataset, val_dataset = random_split(full_train, [train_size, val_size])


    test_dataset = datasets.CIFAR10(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )

  train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  val_loader   = DataLoader(val_dataset, batch_size=256, shuffle=False)
  test_loader  = DataLoader(test_dataset, batch_size=256, shuffle=False)
  return train_loader, val_loader, test_loader

BATCH_SIZE = 128
LR = 0.01
EPOCHS = 20
SEED = 42
MAX_GRAD_NORM = 1.0
DELTA = 1e-5
PATIENCE = 5

## CIFAR-10 Loader

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


full_train = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

train_size = 40000
val_size = len(full_train) - train_size
train_dataset, val_dataset = random_split(full_train, [train_size, val_size])


test_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=256, shuffle=False)

100%|██████████| 170M/170M [00:07<00:00, 23.7MB/s]


## MNIST Loader

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

full_train = datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

train_size = 50000
val_size = len(full_train) - train_size
train_dataset, val_dataset = random_split(full_train, [train_size, val_size])

# FOR TESTING CAN USE A SMALLER SAMPLE SIZE
# train_size = 5000
# val_size = 1000
# subset_train, subset_rest = random_split(full_train, [train_size, len(full_train) - train_size])
# subset_val, _ = random_split(subset_rest, [val_size, len(subset_rest) - val_size])

test_dataset = datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=256, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 39.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.06MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.45MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.13MB/s]


# Baseline CNN Model

This model is chosen in `run_experiment` if an invalid dataset is selected. I should probably replace this with a `ThrowException` instead though.

In [6]:
# baseline model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            MNIST Settings
        #=======================================#

        # self.conv1 = nn.Conv2d(1, 16, 3, 1)
        # self.conv2 = nn.Conv2d(16, 32, 3, 1)
        # self.fc1 = nn.Linear(32*12*12, 64)
        # self.fc2 = nn.Linear(64, 10)

        #=======================================#
        #            CIFAR-10 Settings
        #=======================================#

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=576, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):
        # x = F.relu(self.conv1(x))
        # x = F.relu(self.conv2(x))
        # x = F.max_pool2d(x, 2)
        # x = torch.flatten(x, 1)
        # x = F.relu(self.fc1(x))
        # return self.fc2(x)

        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## Model Architectures
MNIST and CIFAR-10 require different NN structures, so they have their own separate classes that get initialized in `run_experiment`.

In [7]:
# MNIST Model
class CNN_MNIST(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            MNIST Settings
        #=======================================#

        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32*12*12, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


# CIFAR-10 Model

class CNN_CIFAR10(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            CIFAR-10 Settings
        #=======================================#

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=576, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):

        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Model Training
Courtesy of Megha

In [8]:
def train_one_epoch(model, loader, optimizer):
    # train loop
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.cross_entropy(out, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [9]:
@torch.no_grad()
def evaluate(model, loader):
    # evaluation
    model.eval()
    loss, correct = 0, 0
    for data, target in loader:
        data, target = data.to(device), target.to(device)
        out = model(data)
        loss += F.cross_entropy(out, target, reduction="sum").item()
        pred = out.argmax(1)
        correct += pred.eq(target).sum().item()
    loss /= len(loader.dataset)
    acc = 100. * correct / len(loader.dataset)
    return loss, acc

In [10]:
"""
dataset: either MNIST or CIFAR
I set dataset_name as a hyperparameter we can use for convenience and easy swapping
Otherwise, it defaults to MNIST, and invalid datasets give the base CNN model
This makes the dataset selection a modular component, should we want to choose other datasets
"""

def run_experiment(dataset='MNIST', use_dp=False, fixed_privacy_budget=False):

    # setup model and optimizer
    if dataset == 'MNIST':
      model = CNN_MNIST().to(device)
    elif dataset == "CIFAR":
      model = CNN_CIFAR10().to(device)

    # Defaults to base CNN class if no dataset is specified
    else:
      model = CNN().to(device)

    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

    privacy_engine = None
    if use_dp:

        if fixed_privacy_budget:

            privacy_engine = PrivacyEngine()

            # Calculates sigma based on target epsilon and delta
            model, optimizer, train_loader_dp = privacy_engine.make_private_with_epsilon(
              module=model,
              optimizer=optimizer,
              data_loader=train_loader,
              max_grad_norm=MAX_GRAD_NORM,
              target_delta=TARGET_DELTA,
              target_epsilon=TARGET_EPSILON,
              epochs=NUM_EPOCHS
        )
        else:

            privacy_engine = PrivacyEngine()

            # Otherwise takes sigma as a hyperparameter
            model, optimizer, train_loader_dp = privacy_engine.make_private(
              module=model,
              optimizer=optimizer,
              data_loader=train_loader,
              max_grad_norm=MAX_GRAD_NORM,
              noise_multiplier=NOISE_MULTIPLIER,
        )
    else:
        train_loader_dp = train_loader


    # train
    best_val_acc = 0
    epochs_no_improve = 0
    best_model_path = f"best_model{'_dp' if use_dp else ''}.pt"

    start_time = time.time()

    for epoch in range(1, EPOCHS + 1):
        train_loss = train_one_epoch(model, train_loader_dp, optimizer)
        val_loss, val_acc = evaluate(model, val_loader)

        if use_dp:
          eps = privacy_engine.get_epsilon(DELTA)
        else:
          eps = None

        print(f"[{'DP-SGD' if use_dp else 'Standard SGD'}] Epoch {epoch}: "
            + f"train_loss={train_loss:.4f}, "
            + f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}%"
            + f", ε={eps:.4f}" if use_dp else "")

        # if fixed_privacy_budget:
        #   eps = privacy_engine.get_epsilon(TARGET_DELTA)
        #   print(f", ε={eps:.4f}")

        # elif use_dp:
        #   eps = privacy_engine.get_epsilon(DELTA)
        #   print()

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            epochs_no_improve = 0
            torch.save(model.state_dict(), best_model_path)
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= PATIENCE:
            print(f"\nEarly stopping at epoch {epoch} (no improvement for {PATIENCE} epochs).")
            break


    end_time = time.time()
    total_time = end_time - start_time

    # test on best model
    model.load_state_dict(torch.load(best_model_path))
    test_loss, test_acc = evaluate(model, test_loader)
    final_eps = privacy_engine.get_epsilon(DELTA) if use_dp else None

    print(f"\n[{ 'DP-SGD' if use_dp else 'Standard-SGD'}]")
    print(f"Best val acc: {best_val_acc:.4f}%, Test acc: {test_acc:.4f}%\n" + (f" ε={final_eps:.4f}\n" if use_dp else "") + (f"Total runtime: {total_time:.4f} seconds"))
    return best_val_acc, test_acc, final_eps

# Initial Benchmarks
## Model Hyperparameters

In [11]:
BATCH_SIZE = 128
LR = 0.01
EPOCHS = 20
SEED = 42
MAX_GRAD_NORM = 1.0
DELTA = 1e-5
PATIENCE = 5

DATASET_NAME = "CIFAR"
train_loader, val_loader, test_loader = select_dataset(DATASET_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# For dynamically calculated epsilon
NOISE_MULTIPLIER = 1.0

# For a fixed privacy budget (eps, del)
TARGET_EPSILON = 8
TARGET_DELTA = 1e-5
NUM_EPOCHS = 20

torch.manual_seed(SEED)
np.random.seed(SEED)

## Baseline: No Differential Privacy

In [12]:
print("Running standard SGD baseline")
run_experiment(dataset=DATASET_NAME, use_dp=False)


Running standard SGD baseline


KeyboardInterrupt: 

## Baseline: Differential Privacy

In [ ]:
print("\nRunning DP-SGD baseline")
run_experiment(use_dp=True)

## Baseline: Fixed Privacy Budget

In [13]:
print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)


Running DP-SGD baseline for fixed (8, 1e-05)-DP


/usr/local/lib/python3.12/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/tmp/ipython-input-1715503071.py:10: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()


[DP-SGD] Epoch 1: train_loss=2.1560, val_loss=2.0394, val_acc=24.4700%, ε=3.5278
[DP-SGD] Epoch 2: train_loss=2.0040, val_loss=1.9694, val_acc=28.9900%, ε=4.0227
[DP-SGD] Epoch 3: train_loss=1.9391, val_loss=1.9250, val_acc=31.6100%, ε=4.3957
[DP-SGD] Epoch 4: train_loss=1.9009, val_loss=1.8958, val_acc=33.6900%, ε=4.7147
[DP-SGD] Epoch 5: train_loss=1.8676, val_loss=1.8667, val_acc=34.8300%, ε=5.0010
[DP-SGD] Epoch 6: train_loss=1.8529, val_loss=1.8485, val_acc=35.8100%, ε=5.2647
[DP-SGD] Epoch 7: train_loss=1.8373, val_loss=1.8269, val_acc=36.8900%, ε=5.5113
[DP-SGD] Epoch 8: train_loss=1.8068, val_loss=1.8332, val_acc=37.2200%, ε=5.7445
[DP-SGD] Epoch 9: train_loss=1.8065, val_loss=1.7997, val_acc=38.6800%, ε=5.9666
[DP-SGD] Epoch 10: train_loss=1.7958, val_loss=1.8167, val_acc=38.9400%, ε=6.1795
[DP-SGD] Epoch 11: train_loss=1.7766, val_loss=1.8145, val_acc=39.7200%, ε=6.3845
[DP-SGD] Epoch 12: train_loss=1.7727, val_loss=1.7973, val_acc=40.3400%, ε=6.5826
[DP-SGD] Epoch 13: train_

(44.09, 45.15, np.float64(7.993863084259577))

# Further (ε, δ)-DP Testing



## Hyperparameters

In [ ]:
# Constants
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 128
LR = 0.01
EPOCHS = 20
SEED = 42
MAX_GRAD_NORM = 2.0
DELTA = 1e-5
PATIENCE = 5

# Default Values
# For dynamically calculated epsilon
NOISE_MULTIPLIER = 1.0

# For a fixed privacy budget (eps, del)
TARGET_EPSILON = 8
TARGET_DELTA = 1e-5
NUM_EPOCHS = 20

torch.manual_seed(SEED)
np.random.seed(SEED)

epsilon_range = [0.5, 1, 2, 4, 8]
noise_mult_range = [1, 2, 4, 8, 16]


## DP-SGD: Fixed Privacy Budget

In [ ]:
dp_sgd_fixed_test_accs = []

for eps in epsilon_range:
  TARGET_EPSILON = eps
  print(f"\nRunning DP-SGD baseline for fixed ({eps}, 1e-5)-DP")
  _, test_acc, _ = run_experiment(use_dp=True, fixed_privacy_budget=True)
  dp_sgd_fixed_test_accs.append(test_acc)

## DP-SGD: Fixed Noise Multiplier

In [ ]:
dp_sgd_fixed_noise_accs = []
dp_sgd_fixed_noise_eps = []

for sig in noise_mult_range:
  NOISE_MULTIPLIER = sig
  print(f"\nRunning DP-SGD baseline for fixed noise")
  _, test_acc, final_eps = run_experiment(use_dp=True, fixed_privacy_budget=False)
  dp_sgd_fixed_noise_accs.append(test_acc)
  dp_sgd_fixed_noise_eps.append(sig)